In [1]:
import pandas as pd
from scipy.optimize import curve_fit
from scipy.integrate import quad
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Reading in the data

In [13]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True) #to get rid of the spikes in the beginning

#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column

#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    #max value of "Elapsed time" in baseline MID
    masses = [2,12,15,18,28,32,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        print(timestamp)
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)
#time["Elapsed time, mass 40"].describe()

2    2023-03-01 16:02:52.912
3    2023-03-01 16:03:00.687
4    2023-03-01 16:03:08.462
5    2023-03-01 16:03:16.237
6    2023-03-01 16:03:24.612
7    2023-03-01 16:03:32.587
8    2023-03-01 16:03:40.362
9    2023-03-01 16:03:48.137
10   2023-03-01 16:03:55.913
11   2023-03-01 16:04:03.688
12   2023-03-01 16:04:11.463
13   2023-03-01 16:04:19.238
14   2023-03-01 16:04:27.413
15   2023-03-01 16:04:35.588
16   2023-03-01 16:04:43.763
17   2023-03-01 16:04:51.538
18   2023-03-01 16:04:59.313
19   2023-03-01 16:05:07.088
20   2023-03-01 16:05:14.863
21   2023-03-01 16:05:22.638
22   2023-03-01 16:05:30.413
23   2023-03-01 16:05:38.188
24   2023-03-01 16:05:46.363
25   2023-03-01 16:05:54.138
26   2023-03-01 16:06:01.913
27   2023-03-01 16:06:10.088
28   2023-03-01 16:06:17.863
29   2023-03-01 16:06:26.038
30   2023-03-01 16:06:34.213
31   2023-03-01 16:06:41.988
32   2023-03-01 16:06:49.763
Name: mass 2.0_time, dtype: datetime64[ns]
2    2023-03-01 16:02:56.513
3    2023-03-01 16:03:04.288


,mass 2.0_time,mass 12.0_time,mass 15.0_time,mass 18.0_time,mass 28.0_time,mass 32.0_time,mass 44.0_time,"Elapsed time, mass 2","Elapsed time, mass 12","Elapsed time, mass 15","Elapsed time, mass 18","Elapsed time, mass 28","Elapsed time, mass 32","Elapsed time, mass 44"
2,2023/03/01 16:02:52.912,2023/03/01 16:02:56.513,2023/03/01 16:02:57.113,2023/03/01 16:02:57.714,2023/03/01 16:02:58.314,2023/03/01 16:02:58.914,2023/03/01 16:03:00.016,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,2023/03/01 16:03:00.687,2023/03/01 16:03:04.288,2023/03/01 16:03:04.888,2023/03/01 16:03:05.489,2023/03/01 16:03:06.089,2023/03/01 16:03:06.689,2023/03/01 16:03:07.791,7.775,7.775,7.775,7.775,7.775,7.775,7.775
4,2023/03/01 16:03:08.462,2023/03/01 16:03:12.063,2023/03/01 16:03:12.663,2023/03/01 16:03:13.264,2023/03/01 16:03:13.864,2023/03/01 16:03:14.465,2023/03/01 16:03:15.566,15.550,15.550,15.550,15.550,15.550,15.551,15.550
5,2023/03/01 16:03:16.237,2023/03/01 16:03:19.838,2023/03/01 16:03:20.438,2023/03/01 16:03:21.039,2023/03/01 16:03:21.639,2023/03/01 16:03:22.840,2023/03/01 16:03:23.941,23.325,23.325,23.325,23.325,23.325,23.926,23.925
6,2023/03/01 16:03:24.612,2023/03/01 16:03:28.213,2023/03/01 16:03:28.813,2023/03/01 16:03:29.414,2023/03/01 16:03:30.014,2023/03/01 16:03:30.815,2023/03/01 16:03:31.916,31.700,31.700,31.700,31.700,31.700,31.901,31.900
7,2023/03/01 16:03:32.587,2023/03/01 16:03:36.188,2023/03/01 16:03:36.788,2023/03/01 16:03:37.389,2023/03/01 16:03:37.989,2023/03/01 16:03:38.589,2023/03/01 16:03:39.691,39.675,39.675,39.675,39.675,39.675,39.675,39.675
8,2023/03/01 16:03:40.362,2023/03/01 16:03:43.963,2023/03/01 16:03:44.563,2023/03/01 16:03:45.164,2023/03/01 16:03:45.764,2023/03/01 16:03:46.364,2023/03/01 16:03:47.466,47.450,47.450,47.450,47.450,47.450,47.450,47.450
9,2023/03/01 16:03:48.137,2023/03/01 16:03:51.739,2023/03/01 16:03:52.339,2023/03/01 16:03:52.940,2023/03/01 16:03:53.541,2023/03/01 16:03:54.141,2023/03/01 16:03:55.242,55.225,55.226,55.226,55.226,55.227,55.227,55.226
10,2023/03/01 16:03:55.913,2023/03/01 16:03:59.514,2023/03/01 16:04:00.114,2023/03/01 16:04:00.715,2023/03/01 16:04:01.316,2023/03/01 16:04:01.916,2023/03/01 16:04:03.017,63.001,63.001,63.001,63.001,63.002,63.002,63.001
11,2023/03/01 16:04:03.688,2023/03/01 16:04:07.289,2023/03/01 16:04:07.889,2023/03/01 16:04:08.490,2023/03/01 16:04:09.090,2023/03/01 16:04:09.691,2023/03/01 16:04:10.792,70.776,70.776,70.776,70.776,70.776,70.777,70.776


In [3]:
# #read in the baseline MID
# basel = pd.read_csv("baseline MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
# basel.drop([0,1], inplace=True)

# #bring the time data into a new df
# time1 = pd.DataFrame(basel.iloc[:,0::2]) #- get every second column

# #add elapsed time columns
# def elapsed_time(df):
#     itr = 0
#     counter = len(df.columns)
#     masses = [2,12,15,18,28,32,44]
#     for i in df:
#         timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
#         runtime = (timestamp-timestamp[2]).dt.total_seconds()
#         colname = "Elapsed time, mass {}".format(masses[itr]) 
#         df.insert(counter, colname, runtime)
#         itr += 1
#         counter += 1
#     return(df)
# elapsed_time(time1)

# #time["Elapsed time, mass 40"].describe()

In [4]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)
#creating a new df by dropping the rows where system was warm
cut_vaclog = vaclog.loc[vaclog["Elapsed time"] >= 3310]
cut_vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
486,NaN,01/03/2023 17:02:57,3316.0,77.485,1.820000e-09,2.360000e-09,4.996000e-09,3.786000e-07,1.1,245.0,-5.275,4297.626,NaN,NaN
487,NaN,01/03/2023 17:03:04,3323.0,77.485,1.810000e-09,2.350000e-09,4.996000e-09,3.786000e-07,1.1,245.0,-5.163,4297.514,NaN,NaN
488,NaN,01/03/2023 17:03:11,3330.0,77.485,1.810000e-09,2.340000e-09,4.996000e-09,3.786000e-07,1.1,245.0,-5.050,4297.514,NaN,NaN
489,NaN,01/03/2023 17:03:17,3336.0,77.485,1.810000e-09,2.350000e-09,4.996000e-09,3.786000e-07,1.1,245.0,-5.050,4297.626,NaN,NaN
490,NaN,01/03/2023 17:03:24,3343.0,77.485,1.810000e-09,2.350000e-09,4.996000e-09,3.790000e-07,1.1,245.0,-5.275,4297.514,NaN,NaN


### Data processing

In [5]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [6]:
# #Combining the time df's
# t_comb = pd.concat([time1,time],join="inner")
# #t_comb.drop([0,1,91,92], inplace=True)

# t_comb

In [7]:
# #Combining the MIDs
# #read in the baseline MID
# comb = pd.read_csv("combined MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
# comb.drop([0,1,91,92], inplace=True)
# print(comb)


### Plotting

In [8]:
#Plotting the all the relevant MID spectrums

plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(time['Elapsed time, mass 15'],mid["mass 15.0_value"],marker=".", markersize=4,label='M15')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mass fractions", loc="upper right")
plt.title("S3 TDS:continuous injection, ads. isotherm , EGA fully in")

plt.yscale('log')

In [9]:
# #Plotting the baseline mid

# plt.figure(figsize=(14,8))
# plt.plot(time1['Elapsed time, mass 2'],basel["mass 2.0_value"],marker=".", markersize=4,label='M2')
# plt.plot(time1['Elapsed time, mass 12'],basel["mass 12.0_value"],marker=".", markersize=4,label='M12')
# plt.plot(time1['Elapsed time, mass 15'],basel["mass 15.0_value"],marker=".", markersize=4,label='M15')
# plt.plot(time1['Elapsed time, mass 18'],basel["mass 18.0_value"],marker=".", markersize=4,label='M18')
# plt.plot(time1['Elapsed time, mass 28'],basel["mass 28.0_value"],marker=".", markersize=4,label='M28')
# plt.plot(time1['Elapsed time, mass 32'],basel["mass 32.0_value"],marker=".", markersize=4,label='M32')
# plt.plot(time1["Elapsed time, mass 44"],basel["mass 44.0_value"],marker=".", markersize=4,label='M44')

# plt.xlabel('Elapsed time (s)')
# plt.ylabel('Ion current (A)')
# plt.legend(title="Mol masses", loc="upper right")
# plt.title("TDS MID baseline scan no pre-injection, dynamic vacuum")

# plt.yscale('log')

In [10]:
# #Plotting the combined MID
# plt.figure(figsize=(14,8))
# plt.plot(t_comb['Elapsed time, mass 2'],comb["mass 2.0_value"],marker=".", markersize=4,label='M2')
# plt.plot(t_comb['Elapsed time, mass 12'],comb["mass 12.0_value"],marker=".", markersize=4,label='M12')
# plt.plot(t_comb['Elapsed time, mass 15'],comb["mass 15.0_value"],marker=".", markersize=4,label='M15')
# plt.plot(t_comb['Elapsed time, mass 18'],comb["mass 18.0_value"],marker=".", markersize=4,label='M18')
# plt.plot(t_comb['Elapsed time, mass 28'],comb["mass 28.0_value"],marker=".", markersize=4,label='M28')
# plt.plot(t_comb['Elapsed time, mass 32'],comb["mass 32.0_value"],marker=".", markersize=4,label='M32')
# plt.plot(t_comb["Elapsed time, mass 44"],comb["mass 44.0_value"],marker=".", markersize=4,label='M44')

# plt.xlabel('Elapsed time (s)')
# plt.ylabel('Ion current (A)')
# plt.legend(title="Mol masses", loc="upper right")
# plt.title("TDS Combined baseline and MID scans no pre-injection, dynamic vacuum")

# plt.yscale('log')

In [11]:
#Plotting BA2, BA1, DUAL pressure
plt.figure(figsize=(10,6))
plt.plot(cut_vaclog["Elapsed time"],cut_vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(cut_vaclog["Elapsed time"],cut_vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(cut_vaclog["Elapsed time"],cut_vaclog["DUAL experiment"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S3 TDS:continuous injection, ads. isotherm , EGA fully in")
ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="both", which='both')
plt.xlim(0,450000)
plt.grid(which='major', axis='x', markevery="50000")
plt.grid(which='both', axis='y')

plt.show()

In [12]:
# Plotting the temp evolution exc warm part
plt.figure()
plt.plot((cut_vaclog["Elapsed time"]),polyfit(A,cut_vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S3 TDS:continuous injection, ads. isotherm , EGA fully in")
plt.xlim(0,450000)
plt.ylim(0,320)
plt.yscale('linear')